In [1]:
import sys
sys.path.insert(1, '../')

In [2]:
import itertools
import os

import jax
import matplotlib.pyplot as plt
import numpy as np
import jax.numpy as jnp

from jax import config
from scipy import optimize

from vibrojet.basis_utils2 import ContrBasis, HermiteBasis, AssociateLegendreBasis, FourierBasis, generate_prod_ind

from vibrojet.jet_prim import acos
from vibrojet.potentials import nh3_POK
from vibrojet.taylor import deriv_list
from vibrojet.keo import Gmat, com, pseudo

plt.rcParams.update(
    {"text.usetex": True, "font.family": "serif", "font.serif": ["Computer Modern"]}
)

config.update("jax_enable_x64", True)

In [3]:
@jax.jit
def poten(q):
    r1, r2, r3, s4, s5, tau = q
    rho = tau + np.pi / 2

    beta1 = jnp.sqrt(6) / 3 * s4 + 2 * np.pi / 3
    beta2 = -1 / jnp.sqrt(6) * s4 + 1 / jnp.sqrt(2) * s5 + 2 * np.pi / 3
    beta3 = -1 / jnp.sqrt(6) * s4 - 1 / jnp.sqrt(2) * s5 + 2 * np.pi / 3

    cosrho = jnp.cos(rho)
    sinrho = jnp.sin(rho)
    cosrho2 = cosrho * cosrho
    sinrho2 = sinrho * sinrho

    cosalpha2 = cosrho2 + sinrho2 * jnp.cos(beta2)
    cosalpha3 = cosrho2 + sinrho2 * jnp.cos(beta3)
    cosalpha1 = cosrho2 + sinrho2 * jnp.cos(beta2 + beta3)
    alpha1 = acos(cosalpha1)
    alpha2 = acos(cosalpha2)
    alpha3 = acos(cosalpha3)
    v = nh3_POK.poten((r1, r2, r3, alpha1, alpha2, alpha3))
    return v


vmin = optimize.minimize(poten, [1.1, 1.1, 1.1, 0.1, 0.1, 0.1])
q0 = vmin.x
v0 = vmin.fun

print("equilibrium coordinates:", q0)
print("min of the potential:", v0)

equilibrium coordinates: [1.01159999e+00 1.01159999e+00 1.01159999e+00 6.95693777e-09
 2.12708155e-09 3.85722364e-01]
min of the potential: 3.4875100712457326e-11


Find equilibrium values of internal coordinates

In [13]:
print("Reference values of internal coordinates:\n", q0)

Reference values of internal coordinates:
 [1.01159999e+00 1.01159999e+00 1.01159999e+00 6.95693777e-09
 2.12708155e-09 3.85722364e-01]


In [14]:
# masses of N, H1, H2, H3
masses = [14.00307400, 1.007825035, 1.007825035, 1.007825035]

# internal-to-Cartesian coordinate transformation
@com(masses)
def internal_to_cartesian(internal_coords):
    r1, r2, r3, s4, s5, tau = internal_coords
    rho = tau + np.pi / 2

    beta1 = jnp.sqrt(6) / 3 * s4 + 2 * np.pi / 3
    beta2 = -1 / jnp.sqrt(6) * s4 + 1 / jnp.sqrt(2) * s5 + 2 * np.pi / 3
    beta3 = -1 / jnp.sqrt(6) * s4 - 1 / jnp.sqrt(2) * s5 + 2 * np.pi / 3

    cos_rho = jnp.cos(rho)
    sin_rho = jnp.sin(rho)
    cos_beta2 = jnp.cos(beta2)
    cos_beta3 = jnp.cos(beta3)
    sin_beta2 = jnp.sin(beta2)
    sin_beta3 = jnp.sin(beta3)

    xyz = jnp.array(
        [
            [0.0, 0.0, 0.0],
            [r1 * sin_rho, 0.0, r1 * cos_rho],
            [r2 * sin_rho * cos_beta3, r2 * sin_rho * sin_beta3, r2 * cos_rho],
            [r3 * sin_rho * cos_beta2, -r3 * sin_rho * sin_beta2, r3 * cos_rho],
        ]
    )
    return xyz

In [15]:
xyz = internal_to_cartesian(q0)
print("Reference values of internal coordinates:\n", q0)
print("Reference values of Cartesian coordinates:\n", xyz)

Reference values of internal coordinates:
 [1.01159999e+00 1.01159999e+00 1.01159999e+00 6.95693777e-09
 2.12708155e-09 3.85722364e-01]
Reference values of Cartesian coordinates:
 [[-2.61354318e-10 -2.97060229e-10  6.75834408e-02]
 [ 9.37274605e-01 -2.97060229e-10 -3.13009332e-01]
 [-4.68637300e-01  8.11703622e-01 -3.13009333e-01]
 [-4.68637301e-01 -8.11703618e-01 -3.13009331e-01]]


Generate expansion power indices

In [16]:
ncoo = len(q0)
max_order = 2
powers = [np.arange(max_order + 1)] * ncoo
deriv_ind, deriv_mind = next(
    generate_prod_ind(powers, select=lambda ind: np.sum(ind) <= max_order)
)

print("max expansion power:", max_order)
print("number of expansion terms:", len(deriv_ind))

max expansion power: 2
number of expansion terms: 28


Generate expansion of PES in terms of internal coordinates

In [18]:
poten_file = f"nh3_poten_coefs_{max_order}.npy"
if os.path.exists(poten_file):
    print(f"load potential expansion coefs from file {poten_file}")
    poten_coefs = np.load(poten_file)
else:
    print(f"calculate potential expansion coefs and save to {poten_file}")
    poten_coefs = deriv_list(poten, deriv_ind, q0, if_taylor=True)
    np.save(poten_file, poten_coefs)

load potential expansion coefs from file nh3_poten_coefs_2.npy


In [19]:
gmat_file = f"nh3_gmat_coefs_{max_order}.npy"
if os.path.exists(gmat_file):
    print(f"load G-matrix expansion coefs from file {gmat_file}")
    gmat_coefs = np.load(gmat_file)
else:
    print(f"calculate G-matrix expansion coefs and save to {gmat_file}")
    gmat_coefs = deriv_list(
        lambda x: Gmat(x, masses, internal_to_cartesian), deriv_ind, q0, if_taylor=True
    )
    np.save(gmat_file, gmat_coefs)


load G-matrix expansion coefs from file nh3_gmat_coefs_2.npy


In [38]:
mask = deriv_ind != 0
ind0 = np.where(mask.sum(axis=1) == 0)[0][0]
mu = np.diag(gmat_coefs[ind0])[:ncoo]

ind2 = np.array(
    [
        np.where((mask.sum(axis=1) == 1) & (deriv_ind[:, icoo] == 2))[0][0]
        for icoo in range(ncoo)
    ]
)
freq = poten_coefs[ind2] * 2

lin_a = jnp.sqrt(jnp.sqrt(mu / freq))
lin_b = q0

list_herm = [0,1,2]
list_leg = [3,4]
list_fourier = [5]

limits_r = [[0,jnp.inf],[0,jnp.inf],[0,jnp.inf],[-2.5,3.5],[-2.5,2.5],[-1.2,1.2]]

lin_a = jnp.array([jnp.sqrt(jnp.sqrt(mu[i] / freq[i])) if i in list_herm
                    else (limits_r[i][1]-limits_r[i][0])/np.pi if i in list_leg
                    else (limits_r[i][1]-limits_r[i][0])/(2*np.pi)
                    for i in range(ncoo)])

lin_b = jnp.array([q0[i] if i in list_herm
                    else limits_r[i][0] if i in list_leg
                    else limits_r[i][0]
                    for i in range(ncoo)])

print("x->r linear mapping parameters 'a':", lin_a)
print("x->r linear mapping parameters 'b':", lin_b)

# x->r linear mapping function
x_to_r_map = lambda x, icoo: lin_a[icoo] * x + lin_b[icoo]
jac_x_to_r_map = lambda x, icoo: np.ones_like(x) * lin_a[icoo]

#Functions for change of coordinates for Taylor expansion
r_to_y_pes = lambda x, icoo: x - q0[icoo]
r_to_y_gmat = lambda x, icoo: x - q0[icoo]



x->r linear mapping parameters 'a': [0.10007533 0.10007533 0.10007533 1.90985932 1.59154943 0.38197186]
x->r linear mapping parameters 'b': [ 1.01159999  1.01159999  1.01159999 -2.5        -2.5        -1.2       ]


In [40]:
nbas = [80] * ncoo
npoints = [101] * ncoo

p0, p1, p2, p3, p4, p5 = [
    HermiteBasis(
        icoo, nbas[icoo], npoints[icoo], lambda x, icoo=icoo: x_to_r_map(x, icoo),
        lambda x, icoo=icoo: r_to_y_pes(x, icoo), lambda x, icoo=icoo: r_to_y_gmat(x, icoo),
        q0[icoo], deriv_ind[:, icoo])
    if icoo in list_herm else
    AssociateLegendreBasis(
        icoo, nbas[icoo], npoints[icoo], lambda x, icoo=icoo: x_to_r_map(x, icoo),
        lambda x, icoo=icoo: r_to_y_pes(x, icoo), lambda x, icoo=icoo: r_to_y_gmat(x, icoo),
        q0[icoo], deriv_ind[:, icoo], m=1)
    if icoo in list_leg else
    FourierBasis(
        icoo, nbas[icoo], npoints[icoo], lambda x, icoo=icoo: x_to_r_map(x, icoo),
        lambda x, icoo=icoo: r_to_y_pes(x, icoo), lambda x, icoo=icoo: r_to_y_gmat(x, icoo),
        q0[icoo], deriv_ind[:, icoo])
    for icoo in range(ncoo)
]


In [41]:
sum_deriv_ind = np.sum(deriv_ind,axis=1)
poten_coefs_1d = lambda icoo: np.array([poten_coefs[i] if deriv_ind[i,icoo]==sum_deriv_ind[i] else 0.0 for i in range(len(sum_deriv_ind))])
gmat_coefs_1d = lambda icoo: np.array([gmat_coefs[i] if deriv_ind[i,icoo]==sum_deriv_ind[i] else np.zeros_like(gmat_coefs[i]) for i in range(len(sum_deriv_ind))])

c0, c1, c2, c3, c4, c5 = [
    ContrBasis(
        (icoo,), (p0, p1, p2, p3, p4, p5), lambda _: True, gmat_coefs_1d(icoo), poten_coefs_1d(icoo), store_int = True,
    )
    for icoo in range(ncoo)
]

Number of states: 80
batch no: 0  out of: 1
iteration time: 0.33
[ 1790.36106604  5371.0831981   8951.80533018 12532.52746224
 16113.24959431 19693.97172638 23274.69385845 26855.41599052
 30436.13812259 34016.86025466]
Saving integrals...
batch no: 0 out of: 1
iteration time: 0.01
Number of states: 80
batch no: 0  out of: 1
iteration time: 0.02
[ 1790.36105196  5371.08315589  8951.80525982 12532.52736375
 16113.24946767 19693.9715716  23274.69367553 26855.41577946
 30436.13788338 34016.85998731]
Saving integrals...
batch no: 0 out of: 1
iteration time: 0.01
Number of states: 80
batch no: 0  out of: 1
iteration time: 0.03
[ 1790.36107778  5371.08323334  8951.80538891 12532.52754447
 16113.24970003 19693.9718556  23274.69401116 26855.41616672
 30436.13832228 34016.86047785]
Saving integrals...
batch no: 0 out of: 1
iteration time: 0.01
Number of states: 80
batch no: 0  out of: 1
iteration time: 0.03
[  842.59554806  2526.18181642  4210.94173487  5896.88108242
  7584.00554605  9272.320721

In [46]:
# couple CH1 and CH2
p_coefs = np.array([1, 1, 1, 1, 1, 1])
pmax = 60
emax_trunc_ = 60000

e0,e1,e2,e3,e4,e5 = c0.enr,c1.enr,c2.enr,c3.enr,c4.enr,c5.enr
e_sum = [e0 - e0[0],e1 - e1[0],e2 - e2[0],e3 - e3[0],e4 - e4[0],e5 - e5[0],]
#Make energy truncation a part of basis generation for speed.
f_e_sum = lambda ind: np.sum(np.array([e_sum[i][ind[i]] for i in range(len(ind))])) < emax_trunc_
f_pmax = lambda ind: np.sum(np.array(ind) * p_coefs[:len(ind)]) <= pmax

c012 = ContrBasis(
    (0, 1, 2),
    (c0, c1, c2, c3, c4, c5),
    f_e_sum or f_pmax,
    gmat_coefs,
    poten_coefs,
    emax=30000,
    batch_size = 1000000,
    store_int = True,
)

e012 = c012.enr
print(e012[0], e012[0:5] - e012[0])

Number of states: 969
batch no: 0  out of: 1
iteration time: 1.86
[ 6484.88743915  9980.66926004 10107.62978074 10108.0491876
 13476.45108094 13603.41160164 13603.8310085  13730.37212234
 13730.7915292  13731.21093606]
Saving integrals...
batch no: 0 out of: 1
iteration time: 1.6
6484.887439149066 [   0.         3495.7818209  3622.74234159 3623.16174845 6991.56364179]


In [52]:
# couple CH1 and CH2
p_coefs = np.array([1, 1, 1, 1, 1, 1])
pmax = 60
emax_trunc_ = 60000

#Make energy truncation a part of basis generation for speed.
f_e_sum = lambda ind: np.sum(np.array([e_sum[i][ind[i]] for i in range(len(ind))])) < emax_trunc_
f_pmax = lambda ind: np.sum(np.array(ind) * p_coefs[:len(ind)]) <= pmax

c34 = ContrBasis(
    (3, 4),
    (c0, c1, c2, c3, c4, c5),
    f_e_sum or f_pmax,
    gmat_coefs,
    poten_coefs,
    emax=30000,
    batch_size = 1000000,
    store_int = True,
)

e34 = c34.enr
print(e34[0], e34[0:5] - e34[0])

Number of states: 662
batch no: 0  out of: 1
iteration time: 0.49
[4669.01333965 6381.29503916 6381.29504496 8087.6684551  8101.91140702
 8101.91140704 9802.38996118 9802.38997276 9830.84209984 9830.89555738]
Saving integrals...
batch no: 0 out of: 1
iteration time: 0.31
4669.0133396497595 [   0.         1712.28169951 1712.28170531 3418.65511545 3432.89806737]


In [53]:
e5 = c5.enr

print('e012',e012[0], e012[0:5] - e012[0])
print('e34',e34[0], e34[0:5] - e34[0])
print('e5',e5[0], e5[0:5] - e5[0])

e012 6484.887439149066 [   0.         3495.7818209  3622.74234159 3623.16174845 6991.56364179]
e34 4669.0133396497595 [   0.         1712.28169951 1712.28170531 3418.65511545 3432.89806737]
e5 545.0191963303099 [   0.         1090.42024668 2179.91669187 3268.48777966 4356.13194445]


In [56]:
# couple all together
p_coefs = np.array([1, 1, 1])
pmax = 300
emax_trunc_ = 10000
e_sum = [e012 - e012[0],e34 - e34[0],e5 - e5[0]]
#Make energy truncation a part of basis generation for speed.
f_e_sum = lambda ind: np.sum(np.array([e_sum[i][ind[i]] for i in range(len(ind))])) < emax_trunc_
f_pmax = lambda ind: np.sum(np.array(ind) * p_coefs[:len(ind)]) <= pmax
c = ContrBasis(
    (0, 1, 2),
    (c012, c34, c5),
    f_e_sum or f_pmax,
    gmat_coefs,
    poten_coefs,
    store_int = False,
    emax=30000,
    batch_size = 100000,
)
e = c.enr
print(e[0], e[0:20] - e[0])


Number of states: 229
batch no: 0  out of: 1
iteration time: 0.6
[ 7579.96128534  8662.02853184  9266.51697334  9266.51838097
  9740.51039791 10371.45781481 10371.4611865  10820.651369
 10905.12238972 10947.97052226]
7579.961285336338 [   0.         1082.0672465  1686.555688   1686.55709563 2160.54911258
 2791.49652948 2791.49990117 3240.69008366 3325.16110439 3368.00923692
 3368.01105746 3537.40217788 3635.34846431 3635.34984348 3898.59015068
 3898.59328554 4323.29137193 4459.81229012 4506.09748471 4506.09853061]


In [ ]:
# x = np.linspace(-np.pi,5,1000)
# q = np.ones(len(x))[:,None]*q0
# q[:,3] = x
# v = poten(q.T)

# plt.plot(x,v)
# plt.ylim([0,50000])
# plt.xlim([-2.5,3.5])

# limits_r = [[0,jnp.inf],[0,jnp.inf],[0,jnp.inf],[-2.5,3.5],[-2.5,2.5],[-1.2,1.2]]